# QUBO formulation
#### Device: Dirac-1

## Introduction


Quadratic Unconstrained Binary Optimization (QUBO) problems are a class of optimization problems where the goal is to maximize or minimize a quadratic objective function without any constraints on the decision variables. Furthermore, QUBO problems serve as fundamental building blocks for more complex optimization models that can address many real-world problems. This tutorial will go through why QUBOs are valuable, what they can do, and how to run such problems on QCi's Dirac entropy computing systems.


## Importance

QUBO encodings tend to naturally arise in quantum settings, or any setting where variable interactions are directly encoded into physical interactions between bits. The reason for this is that physical interactions tend to naturally be two-body, while most constraints in computer science problems do not. For this reason, QUBOs are a natural underlying model to think about when using a wide variety of quantum optimization systems. Since QUBOs are formally NP-hard, it is technically possible to map any optimization problem to a QUBO. Efforts have been made to do so efficiently. For example, [this work](https://link.springer.com/article/10.1007/s10479-022-04634-2) expresses many common problems as QUBOs with the aim of bridging to quantum technology. QUBOs can be used to construct the kinds of linear constraints often found in optimization problems. Since automatic handling of such constraints is often very useful, we have also developed a related model called Quadratic Linearly Constrained Binary Optimization (QLCBO), which is documented in [this tutorial](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/qlcbo-on-dirac). This lesson however will focus on using the underlying unconstrained model.

## Applications

QUBOs are fundamental in various fields, serving as a basis for formulating discrete combinatorial optimization problems. As an NP-hard problem, QUBO finds numerous applications across diverse domains, including machine learning, operations research, finance, chemistry, medicine, machine learning and beyond. A number of our tutorials and use cases use the QUBO representation directly:

* [QBoost: A QUBO Based Binary Classification Method](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/qboost-for-qubo)
* [Max-Cut Demonstration using Dirac-1](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/max-cut-on-dirac)
* [Dimensionality reduction](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/dimensionality-reduction-on-dirac)
* [Set Partitioning](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/set-partitioning-on-dirac) (A QUBO is sent indirectly, via an eqc-models optimization modeling class, SetPartitionModel)

## Mathematical Definition


A Quadratic Unconstrained Binary Optimization (QUBO) defined by second order interactions between binary variables with no constraints, is defined by linear and quadratic terms, but since $x^2 = x$ when $x \in \{0,1\}$, we can simplify the optimization expression as such $f(x) = \sum_{i} \sum_{j} J_{ij} x_i x_j$, where $f: \mathbb{B}^n \rightarrow \mathbb{R}$. Note that the coefficients naturally encode as a square symmetric matrix, so that $f(x) = x^T Q x$, where $Q$ has entries $Q_{ij}$. The goal of the optimization problem is to find the binary vector, $x^{*}$, that minimizes $f(x)$, 
$x^{*} = \mathop{\arg \min}_{x} x^T Q x$. 

## Framework Overview

- `QuadraticModel`: Encodes the QUBO formulation, handling the linear and quadratic coefficients of the problem and preparing it for solvers.
- `Dirac1CloudSolver`: Submits the QUBO problem to QCi's Dirac-1 device for optimization

The following steps outline the modeling and solution process:

1. Define the QUBO problem using a matrix of quadratic coefficients $J$ and a vector of linear coefficients $C$.
2. Initialize the `QuadraticModel` with $C$ and $J$.
3. Set variable constraints (e.g., binary domain).
4. Use `Dirac1CloudSolver` to submit the model and retrieve results.

## Implementation

In [2]:
import numpy as np
from eqc_models.base import QuadraticModel
from eqc_models.solvers import Dirac1CloudSolver
import os

Establish connection with `Dirac1CloudSolver` and QCi's server using your unique token ID and our default API URL.

In [3]:
token = "your_token"
api_url = "https://api.qci-prod.com"
solver = Dirac1CloudSolver(api_token=token, url=api_url)

**Define the QUBO problem:**

Minimize: $x_1^2 + x_1x_2 + x_2^2 - x_1 - 2x_2$

In [5]:
C = np.array([[-1],[-2]]) # Linear coefficents
J = np.array([[1, 1],[1, 1]]) # Quadratic coefficients

**Model Initialization**

Initialize `QuadraticModel` with the linear and quadratic coefficients, and set the variable constraints (binary variables).

In [11]:
model = QuadraticModel(C, J)
model.upper_bound = np.array([1, 1]) # Variables are binary (0 or 1)
print(model.qubo)

QUBO(Q=array([[ 0.,  1.],
       [ 1., -1.]]))


**Job Submission**

`Dirac1CloudSolver` accepts the `QuadraticModel` and submit it as a QUBO job.

In [8]:
response = solver.solve(model, num_samples=10)

2025-01-27 13:09:36 - Dirac allocation balance = 0 s (unmetered)
2025-01-27 13:09:37 - Job submitted: job_id='6797cbe18dd54d8a9ea609f1'
2025-01-27 13:09:37 - QUEUED
2025-01-27 13:09:39 - RUNNING
2025-01-27 13:11:54 - COMPLETED
2025-01-27 13:11:56 - Dirac allocation balance = 0 s (unmetered)


**Extract results**

The `solutions` and `energies` fields in the response contains the binary solutions found by the solver and the corresponding energy for each solution, respectively.

In [9]:
solutions = response["results"]["solutions"]
energies = response["results"]["energies"]

print("Solutions:", solutions)
print("Energies:", energies)

Solutions: [[0, 1]]
Energies: [-1]


## Conclusion

By using the `QuadraticModel` and `Dirac1CloudSolver`, this tutorial demonstrates a streamlined and modular approach to solving QUBO problems. The `QuadraticModel` abstract the encoding of the problem, while `Dirac1CloudSOlver` simplifies the job submission process to QCi's Dirac-1 device. This approach aligns with the modern practices for solving QUBO problems and extends the scalability of quantum-inspired optimization workflows. 

For applications requiring constraints, a higher level of abstraction, such as QLCBO, might be more suitable. You can explore this approach in [our QLCBO tutorial](https://quantumcomputinginc.com/learn/tutorials-and-use-cases/qlcbo-on-dirac).  If you feel confident with QUBOs and are interested in how they are applied without constraints, we encourage you to explore additional lessons linked above. However, if your application involves constraints, proceeding to the QLCBO tutorial would be the logical next step.

Finally, when you're ready, we invite you to try our device with one of your own problems to experience the power and flexibility of quantum-inspired optimization firsthand.